# Longhorn.io - nur für microk8s

Voraussetzungen 
- isccsi aktiveren
- multipathd (für SAN) deaktiveren
- K8s Label für master setzen

In [ ]:
%%bash
sudo apt-get install open-iscsi
sudo modprobe iscsi_tcp
lsmod | grep iscsi_tcp

In [ ]:
%%bash
sudo systemctl stop multipathd
sudo systemctl disable multipathd
sudo systemctl mask multipathd


In [ ]:
%%bash
kubectl label nodes $(kubectl get nodes -o custom-columns=NAME:.metadata.name | awk 'NR==2') node-role.kubernetes.io/master=
kubectl label nodes $(kubectl get nodes -o custom-columns=NAME:.metadata.name | awk 'NR==2') node-role.kubernetes.io/control-plane=      

Prüfen ob alle Bedinungen erfüllt sind:

In [ ]:
%%bash
curl -sSfL https://raw.githubusercontent.com/longhorn/longhorn/v1.8.0/scripts/environment_check.sh | bash

### Installation

Mit `helm` weil wichtige Variablen überschrieben werden können.

**Links**
* [Quick Installation](https://longhorn.io/docs/1.8.0/deploy/install/)

In [ ]:
%%bash
helm repo add longhorn https://charts.longhorn.io
helm repo update

In [ ]:
%%bash
helm install longhorn longhorn/longhorn \
  --namespace longhorn-system --create-namespace \
  --set defaultSettings.kubernetesClusterAutodetectionMethod="custom" \
  --set defaultSettings.kubeletRootDir="/var/snap/microk8s/common/var/lib/kubelet" \
  --set csi.kubeletRootDir="/var/snap/microk8s/common/var/lib/kubelet"


Zum Testen erstellen wir zwei PVC

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: longhorn-rwo
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 5Gi
  storageClassName: longhorn
EOF

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: longhorn-rwx
spec:
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 5Gi
  storageClassName: longhorn
EOF

In [ ]:
%%bash
kubectl get sc,pv,pvc

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: my-longhorn-pod
spec:
  containers:
    - name: my-app
      image: nginx
      volumeMounts:
        - mountPath: "/data"
          name: my-storage
  volumes:
    - name: my-storage
      persistentVolumeClaim:
        claimName: longhorn-rwx
EOF


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: my-longhorn-pod-2
spec:
  containers:
    - name: my-app
      image: nginx
      volumeMounts:
        - mountPath: "/data"
          name: my-storage
  volumes:
    - name: my-storage
      persistentVolumeClaim:
        claimName: longhorn-rwx
EOF

In [ ]:
%%bash
kubectl describe pod my-longhorn-pod

In [ ]:
%%bash
kubectl exec my-longhorn-pod -- mkdir -p /data/t1
kubectl exec my-longhorn-pod -- touch /data/t1/t1.txt
kubectl exec my-longhorn-pod -- find /data


### Longhorn UI


In [ ]:
%%bash
kubectl patch svc longhorn-frontend -n longhorn-system -p '{"spec": {"type": "NodePort"}}'

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":$(kubectl get -n longhorn-system service longhorn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/"